# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [31]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [32]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,blackmans bay,-43.0167,147.3167,50.13,92,6,4.97,AU,1699284434
1,1,ushuaia,-54.8000,-68.3000,55.06,44,40,13.80,AR,1699284349
2,2,shahr-e babak,30.1165,55.1186,58.87,40,5,3.58,IR,1699284435
3,3,stanley,54.8680,-1.6985,49.19,85,40,9.22,GB,1699284435
4,4,ilulissat,69.2167,-51.1000,26.62,46,75,18.41,GL,1699284435


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [33]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.4,
    tiles = "OSM",
    frame_width = 900,
    frame_height = 800)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [72]:
# Narrow down cities that fit criteria and drop any results with null values
narrowed_city_df = city_data_df.loc[
    (city_data_df["Max Temp"] < 90) & (city_data_df["Max Temp"] > 21) \
    & (city_data_df["Wind Speed"] < 2) \
    & (city_data_df["Cloudiness"] == 0)]

# Drop any rows with null values
narrowed_city_df = narrowed_city_df.dropna()

# Display sample data
narrowed_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
55,55,edmundston,47.3737,-68.3251,30.43,63,0,1.14,CA,1699284397
61,61,ixtapa-zihuatanejo,17.6333,-101.5500,86.09,66,0,0.00,MX,1699284450
222,222,tongchuanshi,35.0806,109.0897,44.89,37,0,1.39,CN,1699284496
268,268,fortuna,40.5982,-124.1573,56.66,93,0,0.00,US,1699284508
420,420,mizdah,31.4451,12.9801,86.13,14,0,1.39,LY,1699284546


### Step 3: Create a new DataFrame called `hotel_df`.

In [73]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = narrowed_city_df[["City", "Country", "Lat", "Lng"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Hotel Name
55,edmundston,CA,47.3737,-68.3251,
61,ixtapa-zihuatanejo,MX,17.6333,-101.5500,
222,tongchuanshi,CN,35.0806,109.0897,
268,fortuna,US,40.5982,-124.1573,
420,mizdah,LY,31.4451,12.9801,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [74]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
edmundston - nearest hotel: Station 127
ixtapa-zihuatanejo - nearest hotel: Embarc Zihuatanejo
tongchuanshi - nearest hotel: No hotel found
fortuna - nearest hotel: Comfort Inn & Suites Redwood Country
mizdah - nearest hotel: No hotel found


,City,Country,Lat,Lng,Hotel Name
55,edmundston,CA,47.3737,-68.3251,Station 127
61,ixtapa-zihuatanejo,MX,17.6333,-101.5500,Embarc Zihuatanejo
222,tongchuanshi,CN,35.0806,109.0897,No hotel found
268,fortuna,US,40.5982,-124.1573,Comfort Inn & Suites Redwood Country
420,mizdah,LY,31.4451,12.9801,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [76]:
%%capture --no-display

# Configure the map plot
hotel_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    size = "Humidity",
    scale = 5,
    color = "City",
    alpha = 0.4,
    tiles = "OSM",
    frame_width = 900,
    frame_height = 800,
    hover_cols = ["Hotel Name", "Country"])

# Display the map
hotel_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)